# 市場新聞對行情影響
這篇主要研究新聞與市場價格之間的關係

## 假設
可以做出以下假設來進行驗證
- 新聞的看漲看跌會影響一部分的散戶操作
- 市場價格會直接影響新聞報導

## 目標
期望目標有以下 : 
- 觀察股市新聞與價格之間的關係
- 將新聞量化當成指標來進行操作

## 計畫
此篇研究可以分類成以下幾點 :
1. 新聞數據蒐集
2. 模型分析量化
3. 數據研究證明

### 新聞數據蒐集
- 重點 :

    從各種網站上蒐集新聞資料，範圍尺度該怎麼界定是需要很大的工程，股票市場是由各種各樣的類股組成，每組的權重也不一樣，不能全部都當成參考依據

    因此最好的做法就是將範圍縮小到一支股票的新聞資訊。

- 數據來源 :

    目前有個股新聞且最好抓資料的是 [Goodinfo!](https://goodinfo.tw/tw/index.asp)，因此會以這個網站為基準，這個網站抓的資訊來源是以下幾站
    - 公告訊息
    - EToday新聞雲
    - Anue鉅亨
    - PR Newswire
    - Investing.com

### 模型分析量化
- 重點 :

    利用 AI model 來分析新聞是看漲看跌中立哪一個 label，模型主要有以下兩種
    - 情感分析模型 (sentiment model) : 可以看成理解語意的分類器，只會回傳 label, score，任務單一，比較小型
    - 大型語言模型 (LLM) : 能力取決於模型參數，要使用 API 付費會比較穩定，優點是可解釋性強

- 本篇使用到的模型 :

    下面兩個都是簡體中文訓練的 bert-base 公開 model，目前缺少公開的中文數據集因此很難自己訓練
    - [yiyanghkust/finbert-tone-chinese](https://huggingface.co/yiyanghkust/finbert-tone-chinese/tree/main)
    - [hw2942/bert-base-chinese-finetuning-financial-news-sentiment-v2](https://huggingface.co/hw2942/bert-base-chinese-finetuning-financial-news-sentiment-v2)

# 1. 數據蒐集

In [1]:
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
import requests

def news_supplier_handler(supplier:str, url:str):
    """處理新聞文章"""
    
    content = ""
    
    if supplier=="Anue鉅亨":
        res = requests.get(url)
        soup = BeautifulSoup(res.content, "html.parser")
        raw_content = soup.find("main", id="article-container").find_all("p")
    elif supplier=="Investing.com":
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        service = Service('./chromedriver')
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        raw_content = soup.find("div", class_="article_WYSIWYG__O0uhw article_articlePage__UMz3q text-[18px] leading-8").find_all("p")
    elif supplier=="ETtoday新聞雲":
        res = requests.get(url)
        soup = BeautifulSoup(res.content, "html.parser")
        raw_content = soup.find("div", class_="story").find_all("p")
    elif supplier=="PR Newswire":
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        service = Service('./chromedriver')
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.get(f'https://goodinfo.tw/tw/{url}')
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        raw_content = soup.find("div", class_="b1 r10").find_all("p")
    else: # 公告訊息
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        service = Service('./chromedriver')
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.get(f'https://goodinfo.tw/tw/{url}')
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        raw_content = soup.find("td", style="padding:16px 9px 16px 18px;font-size:11pt;line-height:28px;")
    for i in raw_content:
            content += i.text
    return content

In [20]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

chrome_options = Options()
chrome_options.add_argument("--headless")
service = Service('./chromedriver')
driver = webdriver.Chrome(service=service, options=chrome_options)

df = pd.DataFrame()
df['date'] = None
df['supplier'] = None
df['link'] = None
df['content'] = None

error_index = []

i = 4
url = f"""https://goodinfo.tw/tw/StockAnnounceList.asp?PAGE=1&START_DT=202{i}-01-01&END_DT=202{i}-12-31&STOCK_ID=2330&KEY_WORD=&NEWS_SRC=公告訊息&NEWS_SRC=ETtoday新聞雲&NEWS_SRC=Anue鉅亨&NEWS_SRC=PR+Newswire&NEWS_SRC=Investing.com"""
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
page_count = soup.find("p", style="font-size:11pt;margin-top:4pt;margin-bottom:0pt").text.split("共")[1][1:-3]
for page in range(int(page_count)):
    url = f"""https://goodinfo.tw/tw/StockAnnounceList.asp?PAGE={page+1}&START_DT=202{i}-01-01&END_DT=202{i}-12-31&STOCK_ID=2330&KEY_WORD=&NEWS_SRC=公告訊息&NEWS_SRC=ETtoday新聞雲&NEWS_SRC=Anue鉅亨&NEWS_SRC=PR+Newswire&NEWS_SRC=Investing.com"""
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    news_list = soup.find("div",id="divNewsList").find_all("tr", valign="top")
    for news in news_list:
        news_date = news.find("span", style="font-size:9pt;color:gray;font-weight:normal;").text
        news_date = datetime.strptime(news_date[-17:-7], "%Y/%m/%d")
        news = news.find_all("a")
        link = news[1].get('href')
        news_supplier = news[0].text
        try :
            news_content = news_supplier_handler(news_supplier, link)
            new_row = {'date':news_date, 'supplier':news_supplier, 'link':link, 'content':news_content}
            df.loc[len(df)+1] = new_row
        except:
            new_row = {'date':news_date, 'supplier':news_supplier, 'link':link, 'content':None}
            df.loc[len(df)+1] = new_row
            error_index.append(len(df))

In [23]:
temp_ddd = [3, 4, 6, 12, 13, 201, 341, 1338, 1403, 2088, 2161, 2444]
error_index = [3, 4, 6, 12, 13, 341, 1403, 2161]

In [24]:
for i in error_index:
    print(df.loc[i, 'link'])


https://news.cnyes.com/news/id/5661594?utm_source=RWtaTXdQNjF2MkRWZDBscg==&utm_medium=NewsApi
https://news.cnyes.com/news/id/5661539?utm_source=RWtaTXdQNjF2MkRWZDBscg==&utm_medium=NewsApi
https://news.cnyes.com/news/id/5661087?utm_source=RWtaTXdQNjF2MkRWZDBscg==&utm_medium=NewsApi
https://news.cnyes.com/news/id/5660941?utm_source=RWtaTXdQNjF2MkRWZDBscg==&utm_medium=NewsApi
https://news.cnyes.com/news/id/5660946?utm_source=RWtaTXdQNjF2MkRWZDBscg==&utm_medium=NewsApi
https://hk.investing.com/news/stock-market-news/article-571572
https://hk.investing.com/news/stock-market-news/article-506647
https://hk.investing.com/news/economy/article-463349


In [26]:
def error_page_handle(df:pd.DataFrame, error_index:list):
    error_trying = 0
    while error_index:
        try:
            index = error_index[0]
            supplier = df['supplier'][index]
            link = df['link'][index]
            content = news_supplier_handler(supplier=supplier, url=link)
            df.at[index, 'content'] = content
            print(f"index:{index}   擷取成功")
            error_index.pop(0)
        except:
            if error_trying>=10:
                break
            error_trying+=1
            print(f"index:{index} 擷取失敗 重新嘗試中...")
    return df, error_index

暫存流量限制，因為爬蟲過程漫長且網站有阻擋機制，因此分批處理後做整合

In [25]:
temp_df = df
temp_error_index = error_index

In [27]:
temp_df, error_index = error_page_handle(temp_df, temp_error_index)

index:3   擷取成功
index:4   擷取成功
index:6   擷取成功
index:12   擷取成功
index:13   擷取成功
index:341   擷取成功
index:1403   擷取成功
index:2161   擷取成功


In [29]:
temp_df.to_json("2024_news.json", orient='records', lines=True)

# 2. 模型分析量化

In [4]:
import re

delimiters = "[， 。]"

def split_text_by_length(text, max_length=512):
    words = re.split(delimiters, text)
    split_texts = []
    current_part = []
    current_length = 0

    for word in words:
        if current_length + len(word) + 1 <= max_length:
            current_part.append(word)
            current_length += len(word) + 1
        else:
            split_texts.append(' '.join(current_part))
            current_part = [word]
            current_length = len(word) + 1

    if current_part:
        split_texts.append(' '.join(current_part))

    return split_texts

In [5]:
text = """
1.事實發生日:112/12/08
2.公司名稱:台灣積體電路製造股份有限公司
3.與公司關係(請輸入本公司或子公司):本公司
4.相互持股比例:不適用
5.發生緣由:不適用
6.因應措施:不適用
7.其他應敘明事項(若事件發生或決議之主體係屬公開發行以上公司，本則重大訊息同時
  符合證券交易法施行細則第7條第9款所定對股東權益或證券價格有重大影響之事項):
台積公司今（8）日公佈2023年11月營收報告。2023年11月合併營收約為新台幣
2,060億2,600萬元，較上月減少了15.3%，較去年同期減少了7.5%。累計2023年1至
11月營收約為新台幣1兆9,854億3,600萬元，較去年同期減少了4.1%。"""

text = split_text_by_length(text)

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from typing import Literal
import torch
def sentiment_model_label(model:Literal['finbert-tone-chinese', 'bert-base-chinese-finetuning-financial-news-sentiment-v2']):
    """label news documents with sentiment model

    Args:
        model (str): choose two different model
    """
    model_name = f"../model/{model}"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    device = 0 if torch.cuda.is_available() else -1

    nlp = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device)

    result = nlp(text[0])
    
    return result